In [3]:
import numpy as np
import sisl

def graphene_bands(nkpoints, onsite=0, gamma=-2.7, overlap=0.03):
    """Explain your function in one line
    
    Maybe provide extra info here?
    
    Parameters
    ----------
    Document what each parameter does.
        
    Returns
    -------
    Document what the function returns
    """
    graphene_cell = np.array([
       [2.13,  1.23,  0. ],
       [2.13, -1.23,  0. ],
       [0.  ,  0.  , 14.2]
    ])
    reciprocal_cell = sisl.geom.graphene().rcell
    print(reciprocal_cell)
    
    # The translation vectors to cells that are considered neighbours.
    Rnn = [[0, 0, 0], -graphene_cell[0], -graphene_cell[1]]
    
    # Get all the k values that we are going to use.
    
    # First find the coordinates of the high simmetry points
    M = 2/3 * reciprocal_cell[0] + 1/3 * reciprocal_cell[1]
    K = 1/2 * reciprocal_cell[0]
    
    print(M, K)
    
    # And then build the path
    first_segment = 2 * (nkpoints // 3)
    ks = [*np.linspace(0, M, first_segment), *np.linspace(M, K, nkpoints - first_segment + 1)[1:]]
    
    # Initialize the eigenvalues array, where we will store everything.
    eigenvalues = np.zeros((nkpoints, 2))

    # Then loop through all the k points
    for i, k in enumerate(ks):
        # Initialize our hamiltonian and overlap matrices, just with the diagonal elements
        H_k = np.array([
            [onsite, 0],
            [0, onsite]
        ], dtype=complex)
        S_k = np.array([
            [1, 0],
            [0, 1]
        ], dtype=complex)
        
        # And now, we are going to fill the non-diagonal terms
        # Loop through all cells in the supercell
        for R in Rnn:
            # Calculate the phase value for this translation and K point
            phase = np.exp(- k.dot(R) * 1j)
            # And now add the contribution to the hamiltonian and overlap.
            H_k[0,1] += gamma * phase
            S_k[0,1] += overlap * phase

        # Finally, diagonalize and extract the eigenvalues.
        k_eigenvalues, k_eigenvecs = eigh(H_k, S_k, lower=False)
        # And store them in our array.
        eigenvalues[i, :] = k_eigenvalues

    # We are done looping, just return our results
    return ks, eigenvalues


k, E = graphene_bands(300, 0)

[[ 1.47492613  2.55464699 -0.        ]
 [ 1.47492613 -2.55464699 -0.        ]
 [-0.         -0.          0.44247784]]
[ 1.47492613  0.851549   -0.        ] [ 0.73746306  1.2773235  -0.        ]


NameError: name 'eigh' is not defined

In [7]:
1/np.tan(np.radians(30))

1.7320508075688774

Graphene 3D surface
----

In [ ]:
reciprocal = geom.rcell[:2,:2]

# The translation vectors to cells that are considered neighbours.
Rnn = [[1.42, 0], [-0.71, 1.23], [-0.71, -1.23]]
# Get all the k values that we are going to use.
grid_size = 25
coordinates = np.linspace(0, 1, 25)
X, Y = np.meshgrid(coordinates, coordinates)
nkpoints = np.prod(X.shape)
# Initialize the eigenvalues array, where we will store everything.
eigenvalues = np.zeros((*X.shape, 2))

# We can start looping through all the K points to get the eigenvalues for each of them.
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        # Initialize our hamiltonian and overlap matrices, just with the diagonal elements
        H = np.array([
            [e2p, 0],
            [0, e2p]
        ], dtype=complex)
        S = np.array([
            [1, 0],
            [0, 1]
        ], dtype=complex)
        
        k = X[i,j] * reciprocal[0] + Y[i,j] * reciprocal[1]
        # And now, we are going to fill the non-diagonal term
        # Loop through all cells in the supercell
        for R in Rnn:
            # Calculate the phase value for this translation and K point
            phase = np.exp(- k.dot(R) * 1j)
            # And now add the contribution to the hamiltonian and overlap.
            H[0,1] += gamma0 * phase
            S[0,1] += s0 * phase

        # Finally, diagonalize and extract the eigenvalues.
        k_eigenvalues, k_eigenvecs = eigh(H, S, lower=False)
        # And store them in our array.
        eigenvalues[i, j, :] = k_eigenvalues

In [ ]:
import plotly.graph_objects as go

go.Figure(
    data=[go.Surface(
        x=X, y=Y, z=eigenvalues[..., state], coloraxis="coloraxis", name=f"State {state}"
    ) for state in [0, 1]]
)